In [1]:
from transformers import (pipeline, BertTokenizer, BertForMaskedLM, BertConfig, DataCollatorForLanguageModeling,
                            Trainer, TrainingArguments)
import torch
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets


In [18]:

unmasker = pipeline('fill-mask', model='bert-base-uncased')

unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731099545955658,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.08774477988481522,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.05338403955101967,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.04667219892144203,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello i'm a super model."},
 {'score': 0.02709587849676609,
  'token': 2986,
  'token_str': 'fine',
  'sequence': "hello i'm a fine model."}]

In [19]:
config = BertConfig.from_pretrained('bert-base-uncased')
model_path = "best_model.pt"
state_dict = torch.load(model_path)
model = BertForMaskedLM(config)
model.load_state_dict(state_dict)
model.eval()

RuntimeError: Error(s) in loading state_dict for BertForMaskedLM:
	Missing key(s) in state_dict: "cls.predictions.bias", "cls.predictions.transform.dense.weight", "cls.predictions.transform.dense.bias", "cls.predictions.transform.LayerNorm.weight", "cls.predictions.transform.LayerNorm.bias", "cls.predictions.decoder.weight", "cls.predictions.decoder.bias". 
	Unexpected key(s) in state_dict: "classifier.weight", "classifier.bias", "bert.pooler.dense.weight", "bert.pooler.dense.bias". 

In [ ]:
text = "The quick brown [MASK] jumps over the lazy dog."
tokenized_text = text.split()
tokenized_text[3] = '[MASK]'
masked_text = ' '.join(tokenized_text)

input = tokenizer.encode(masked_text, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:

    print(tokenizer.decode(token))




c a t
d o g
b e a r
m a n
# # i e


In [20]:
from transformers import BertTokenizer, BertForMaskedLM
from sklearn.metrics import accuracy_score
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the model's predictions
predicted_words = []

for sentence in sentences:
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Use the model to predict the masked token
    outputs = model(**inputs)
    predictions = outputs.logits[0, mask_index, :]

    # Get the predicted token
    predicted_token = torch.argmax(predictions)
    predicted_word = tokenizer.decode([predicted_token])

    # Add the predicted word to the list
    predicted_words.append(predicted_word)

# Calculate the accuracy of the model's predictions
accuracy = accuracy_score(actual_words, predicted_words)

print(f"Accuracy: {accuracy}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.0


In [2]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the losses
losses = []

for sentence, actual_word in zip(sentences, actual_words):
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Replace the masked token with the actual token
    inputs["input_ids"][0, mask_index] = tokenizer.encode(actual_word, add_special_tokens=False)[0]

    # Use the model to calculate the loss
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

    # Add the loss to the list
    losses.append(loss.item())

# Calculate the average loss
average_loss = sum(losses) / len(losses)

print(f"Average Loss: {average_loss}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 3.7346343994140625


In [10]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [4]:
wiki_dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
wiki_dataset["train"][4]["text"]

" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n . \n"

In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [7]:
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt")

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


In [8]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator
)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hdasilva. Use `wandb login --relogin` to force relogin


{'eval_loss': 2.280414581298828,
 'eval_runtime': 610.3044,
 'eval_samples_per_second': 7.141,
 'eval_steps_per_second': 0.447}

In [12]:
tokenizer = BertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = BertForMaskedLM.from_pretrained('distilbert/distilbert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=512)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', '

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [13]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator
)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

OutOfMemoryError: CUDA out of memory. Tried to allocate 954.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 1.78 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)